# Mobility data visualiser for pymobility-generated Gauss-Markov location iterators
---
To successfully run this make sure you are in an environment that supports scikit-mobility.

# Imports
---
Cell also includes some baseline settings for static libraries

In [1]:
import numpy as np
import pandas as pd
import skmob
from tqdm import tqdm

tqdm.pandas()

# import seaborn as sns
#
# sns.set(style="white", color_codes=True)
# sns.set_context(rc={"font.family": 'sans', "font:size": 24, "axes.titlesize": 24, "axes.lavelsize": 24})

import warnings

warnings.filterwarnings("ignore")

from pymobility.models.mobility import gauss_markov

# Set metaparameters
---

## Parameters for the Gauss-Markov generator

In [2]:
POP_SIZE = 10
TOTAL_AREA_DIMS = (10000, 10000)
ALPHA = .5
VELOCITY_MEAN = 7.
VARIANCE = 7.
SIM_DURATION = 1440

# Define minutely generator
---

In [3]:
class MinutelyMovement:
    def __init__(self, movement_iter):
        self._move = movement_iter

    def __next__(self):
        for i in range(60):
            self_next = next(self._move)

        return self_next

# Generator creation
---

In [4]:
mobility_generator = gauss_markov(nr_nodes=POP_SIZE,
                                  dimensions=TOTAL_AREA_DIMS,
                                  velocity_mean=VELOCITY_MEAN,
                                  alpha=ALPHA,
                                  variance=VARIANCE)

minutely_gen = MinutelyMovement(movement_iter=mobility_generator)

# Randomize initial locations by running the iterator 500 times

In [5]:
for i in range(500):
    content = next(minutely_gen)

    big_mob_df = pd.DataFrame(content, columns=['x', 'y'])
    big_mob_df.insert(0, "uid", range(len(content)))
    big_mob_df.insert(0, "time_step", np.zeros(len(content), dtype=int))

# Iterate SIM_DURATION times and add to the dataframe

In [6]:
for i in range(SIM_DURATION):
    mob_list = next(minutely_gen)

    aux_df = pd.DataFrame(mob_list, columns=['x', 'y'], index=range((i + 1) * len(mob_list), (i + 2) * len(mob_list)))
    aux_df.insert(0, "uid", range(len(mob_list)))
    aux_df.insert(0, "time_step", np.zeros(len(mob_list), dtype=int) + i)

    frames = [big_mob_df, aux_df]
    big_mob_df = pd.concat(frames)

# Center around mean on both axes

In [7]:
big_mob_df['x'] -= big_mob_df['x'].mean()
big_mob_df['y'] -= big_mob_df['y'].mean()

# Translate metres to latitude/longitude

In [8]:
big_mob_df[['x', 'y']] *= 9.00900901e-6

# Center around Cluj-Napoca's city centre coordinates

In [9]:
big_mob_df['x'] += 46.7712
big_mob_df['y'] += 23.6236

# Produce skmob dataframe

In [10]:
skmob_cluj = skmob.TrajDataFrame(data=big_mob_df,
                                 latitude='x',
                                 longitude='y',
                                 datetime='time_step',
                                 user_id='uid')

In [11]:
skmob_cluj

,datetime,uid,lat,lng
0,1970-01-01 00:00:00.000000000,0,46.745879,23.603078
1,1970-01-01 00:00:00.000000000,1,46.773960,23.624650
2,1970-01-01 00:00:00.000000000,2,46.778206,23.580801
3,1970-01-01 00:00:00.000000000,3,46.794214,23.583699
4,1970-01-01 00:00:00.000000000,4,46.782605,23.655228
...,...,...,...,...
14405,1970-01-01 00:00:00.000001439,5,46.770215,23.643515
14406,1970-01-01 00:00:00.000001439,6,46.778220,23.645619
14407,1970-01-01 00:00:00.000001439,7,46.733733,23.615617
14408,1970-01-01 00:00:00.000001439,8,46.738021,23.654803


# Plot trajectories

In [12]:
skmob_cluj.plot_trajectory()